## LLM Evaluation

In [ ]:
#pip install evaluate

In [ ]:
!pip install evaluate rouge_score --quiet

ERROR: Could not find a version that satisfies the requirement rouge_score--quiet (from versions: none)
ERROR: No matching distribution found for rouge_score--quiet


In [4]:
prediction = ["Today is Wednesday and I am enjoying it here.".lower()]
actual = ["Today is wednesday and I want to travel somwhere.".lower()]

In [5]:
import evaluate
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=prediction,references=actual)
print(results)

{'bleu': 0.4111336169005197, 'precisions': [0.6, 0.4444444444444444, 0.375, 0.2857142857142857], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 10, 'reference_length': 10}


In [10]:
rouge = evaluate.load("rouge")
results = rouge.compute(predictions=prediction,references=actual)
print(results)

{'rouge1': np.float64(0.5555555555555556), 'rouge2': np.float64(0.5), 'rougeL': np.float64(0.5555555555555556), 'rougeLsum': np.float64(0.5555555555555556)}


In [11]:
meteor = evaluate.load("meteor")
results = meteor.compute(predictions=prediction,references=actual)
print(results)

[nltk_data] Downloading package wordnet to /home/user/nltk_data...
[nltk_data] Downloading package punkt_tab to /home/user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /home/user/nltk_data...


{'meteor': np.float64(0.5888888888888889)}


# LLM Evaluation with MLflow Example Notebook

In this notebook, we will demonstrate how to evaluate various LLMs and RAG systems with MLflow, leveraging simple metrics such as toxicity, as well as LLM-judged metrics such as relevance, and even custom LLM-judged metrics such as professionalism

We need to set our OpenAI API key, since we will be using GPT-4 for our LLM-judged metrics.

In order to set your private key safely, please be sure to either export your key through a command-line terminal for your current instance, or, for a permanent addition to all user-based sessions, configure your favored environment management configuration file (i.e., .bashrc, .zshrc) to have the following entry:

`OPENAI_API_KEY=<your openai API key>`

In [1]:
import openai
import pandas as pd
import mlflow

## Basic Question-Answering Evaluation

Create a test case of `inputs` that will be passed into the model and `ground_truth` which will be used to compare against the generated output from the model.

In [2]:
eval_df = pd.DataFrame(
    {
        "inputs": [
            "How does useEffect() work?",
            "What does the static keyword in a function mean?",
            "What does the 'finally' block in Python do?",
            "What is the difference between multiprocessing and multithreading?",
        ],
        "ground_truth": [
            "The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.",
            "Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.",
            "'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.",
            "Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.",
        ],
    }
)
eval_df

,inputs,ground_truth
0,How does useEffect() work?,The useEffect() hook tells React that your com...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th..."
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...


Create a simple OpenAI model that asks gpt-4o to answer the question in two sentences. Call `mlflow.evaluate()` with the model and evaluation dataframe. 

In [3]:
from openai import AzureOpenAI
client = AzureOpenAI()

In [4]:
mlflow.set_experiment("LLM Evaluation")

<Experiment: artifact_location='file:///home/user/Desktop/GAAPB01-training-code-base/mlruns/496554240837618347', creation_time=1749014659746, experiment_id='496554240837618347', last_update_time=1749014659746, lifecycle_stage='active', name='LLM Evaluation', tags={}>

In [17]:
import os
os.environ['OPENAI_API_TYPE']='azure'
os.environ['OPENAI_API_KEY']=os.environ['AZURE_OPENAI_API_KEY']
os.environ['OPENAI_API_BASE']=os.environ['AZURE_OPENAI_ENDPOINT']
os.environ['OPENAI_API_TYPE']='azure'
os.environ['OPENAI_DEPLOYMENT_NAME']='gpt-4.1-mini'

In [23]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question in two sentences"
    basic_qa_model = mlflow.openai.log_model(
        model="gpt-4.1-mini",
        task=client.chat.completions,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",  # specify which column corresponds to the expected output
        model_type="question-answering",  # model type indicates which metrics are relevant for this task
        evaluators="default",
        extra_metrics=[mlflow.metrics.latency(),
                       mlflow.metrics.flesch_kincaid_grade_level(),
                       mlflow.metrics.ari_grade_level(),
                       mlflow.metrics.rougeL(),mlflow.metrics.bleu()]
    )
results.metrics

/home/user/gen-ai/lib/python3.12/site-packages/mlflow/models/model.py:365: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.utc_time_created = str(utc_time_created or datetime.utcnow())
2025/06/04 11:15:23 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://anshuopenai2.opena

{'latency/mean': np.float64(2.234647214412689),
 'latency/variance': np.float64(0.1509704671501133),
 'latency/p90': np.float64(2.6684564828872683),
 'toxicity/v1/mean': np.float64(0.0002589545874798205),
 'toxicity/v1/variance': np.float64(9.482707749030139e-09),
 'toxicity/v1/p90': np.float64(0.0003651319624623284),
 'toxicity/v1/ratio': 0.0,
 'exact_match/v1': 0.0,
 'rougeL/v1/mean': np.float64(0.187327025086728),
 'rougeL/v1/variance': np.float64(0.0012536978725254525),
 'rougeL/v1/p90': np.float64(0.22642706131078225),
 'bleu/v1/mean': np.float64(0.0),
 'bleu/v1/variance': np.float64(0.0),
 'bleu/v1/p90': np.float64(0.0)}

Inspect the evaluation results table as a dataframe to see row-by-row metrics to further assess model performance

In [24]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,latency,token_count,toxicity/v1/score,rougeL/v1/score,bleu/v1/score
0,How does useEffect() work?,The useEffect() hook tells React that your com...,`useEffect()` is a React hook that runs side e...,2.411224,58,0.000384,0.209302,0
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function means that th...,1.781982,47,0.000145,0.157303,0
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used to defin...,1.966684,50,0.000322,0.233766,0
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves running multiple proc...,2.778699,63,0.000186,0.148936,0


## LLM-judged correctness with OpenAI GPT-4

Construct an answer similarity metric using the `answer_similarity()` metric factory function.

In [26]:
from mlflow.metrics.genai import EvaluationExample, answer_similarity

# Create an example to describe what answer_similarity means like for this problem.
example = EvaluationExample(
    input="What is MLflow?",
    output="MLflow is an open-source platform for managing machine "
    "learning workflows, including experiment tracking, model packaging, "
    "versioning, and deployment, simplifying the ML lifecycle.",
    score=4,
    justification="The definition effectively explains what MLflow is "
    "its purpose, and its developer. It could be more concise for a 5-score.",
    grading_context={
        "targets": "MLflow is an open-source platform for managing "
        "the end-to-end machine learning (ML) lifecycle. It was developed by Databricks, "
        "a company that specializes in big data and machine learning solutions. MLflow is "
        "designed to address the challenges that data scientists and machine learning "
        "engineers face when developing, training, and deploying machine learning models."
    },
)

# Construct the metric using OpenAI GPT-4 as the judge
answer_similarity_metric = answer_similarity(model="openai:/gpt-4.1-mini", examples=[example])

print(answer_similarity_metric)

EvaluationMetric(name=answer_similarity, greater_is_better=True, long_name=answer_similarity, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's answer_similarity based on the rubric
justification: Your reasoning about the model's answer_similarity score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_similarity based on the input and output.
A definition of answer_similarity and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them be

Call `mlflow.evaluate()` again but with your new `answer_similarity_metric`

In [27]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[answer_similarity_metric],  # use the answer similarity metric created above
    )
results.metrics

2025/06/04 11:17:24 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025/06/04 11:17:26 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/04 11:17:27 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logg

  0%|          | 0/1 [00:00<?, ?it/s]

2025/06/04 11:17:31 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/06/04 11:17:31 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.
2025/06/04 11:17:31 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for automated readability index metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/06/04 11:17:31 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'ari_grade_level' at index 3 in the `extra_metrics` parameter because it returned None.


  0%|          | 0/4 [00:00<?, ?it/s]

{'toxicity/v1/mean': np.float64(0.0002245521500299219),
 'toxicity/v1/variance': np.float64(4.5523647308538654e-09),
 'toxicity/v1/p90': np.float64(0.00029370783595368267),
 'toxicity/v1/ratio': 0.0,
 'exact_match/v1': 0.0,
 'answer_similarity/v1/mean': np.float64(3.75),
 'answer_similarity/v1/variance': np.float64(1.6875),
 'answer_similarity/v1/p90': np.float64(5.0)}

See the row-by-row LLM-judged answer similarity score and justifications

In [28]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,toxicity/v1/score,answer_similarity/v1/score,answer_similarity/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,useEffect() is a React hook that runs side eff...,56,0.000285,5,The output closely aligns with the target by a...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function means the fun...,58,0.000145,2,The output explains the meaning of the static ...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python contains code th...,46,0.000297,5,The output fully captures the key aspects of t...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves running multiple proc...,59,0.000171,3,The output explains multiprocessing and multit...


## Custom LLM-judged metric for professionalism

Create a custom metric that will be used to determine professionalism of the model outputs. Use `make_genai_metric` with a metric definition, grading prompt, grading example, and judge model configuration

In [29]:
from mlflow.metrics.genai import EvaluationExample, make_genai_metric

professionalism_metric = make_genai_metric(
    name="professionalism",
    definition=(
        "Professionalism refers to the use of a formal, respectful, and appropriate style of communication that is tailored to the context and audience. It often involves avoiding overly casual language, slang, or colloquialisms, and instead using clear, concise, and respectful language"
    ),
    grading_prompt=(
        "Professionalism: If the answer is written using a professional tone, below "
        "are the details for different scores: "
        "- Score 1: Language is extremely casual, informal, and may include slang or colloquialisms. Not suitable for professional contexts."
        "- Score 2: Language is casual but generally respectful and avoids strong informality or slang. Acceptable in some informal professional settings."
        "- Score 3: Language is balanced and avoids extreme informality or formality. Suitable for most professional contexts. "
        "- Score 4: Language is noticeably formal, respectful, and avoids casual elements. Appropriate for business or academic settings. "
        "- Score 5: Language is excessively formal, respectful, and avoids casual elements. Appropriate for the most formal settings such as textbooks. "
    ),
    examples=[
        EvaluationExample(
            input="What is MLflow?",
            output=(
                "MLflow is like your friendly neighborhood toolkit for managing your machine learning projects. It helps you track experiments, package your code and models, and collaborate with your team, making the whole ML workflow smoother. It's like your Swiss Army knife for machine learning!"
            ),
            score=2,
            justification=(
                "The response is written in a casual tone. It uses contractions, filler words such as 'like', and exclamation points, which make it sound less professional. "
            ),
        )
    ],
    version="v1",
    model="openai:/gpt-4o",
    parameters={"temperature": 0.0},
    grading_context_columns=[],
    aggregations=["mean", "variance", "p90"],
    greater_is_better=True,
)

print(professionalism_metric)

EvaluationMetric(name=professionalism, greater_is_better=True, long_name=professionalism, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's professionalism based on the rubric
justification: Your reasoning about the model's professionalism score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called professionalism based on the input and output.
A definition of professionalism and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before complet

Call `mlflow.evaluate` with your new professionalism metric. 

In [30]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],  # use the professionalism metric we created above
    )
print(results.metrics)

2025/06/04 11:17:49 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025/06/04 11:18:02 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/04 11:18:03 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logg

  0%|          | 0/1 [00:00<?, ?it/s]

2025/06/04 11:18:06 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/06/04 11:18:06 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.
2025/06/04 11:18:06 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for automated readability index metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/06/04 11:18:06 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'ari_grade_level' at index 3 in the `extra_metrics` parameter because it returned None.
2025/06/04 11:18:06 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'exact_match' at index 4 in the `extra_metrics` parameter because it returned None.


  0%|          | 0/4 [00:00<?, ?it/s]

{'toxicity/v1/mean': np.float64(0.00023058175429468974), 'toxicity/v1/variance': np.float64(3.722053125755136e-09), 'toxicity/v1/p90': np.float64(0.0002926595218013972), 'toxicity/v1/ratio': 0.0, 'professionalism/v1/mean': np.float64(4.0), 'professionalism/v1/variance': np.float64(0.0), 'professionalism/v1/p90': np.float64(4.0)}


In [31]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,toxicity/v1/score,professionalism/v1/score,professionalism/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React runs side effe...,56,0.000282,4,"The response uses clear, formal, and respectfu..."
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function means that th...,58,0.000151,4,"The response uses clear, formal, and respectfu..."
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python contains code th...,46,0.000297,4,The response uses formal and respectful langua...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves running multiple proc...,56,0.000192,4,"The response uses clear, formal, and respectfu..."


Lets see if we can improve `basic_qa_model` by creating a new model that could perform better by changing the system prompt.

Call `mlflow.evaluate()` using the new model. Observe that the professionalism score has increased!

In [36]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question using extreme formality."
    professional_qa_model = mlflow.openai.log_model(
        model="gpt-4.1-mini",
        task=openai.chat.completions,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        professional_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],
    )
print(results.metrics)

/home/user/gen-ai/lib/python3.12/site-packages/mlflow/models/model.py:365: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.utc_time_created = str(utc_time_created or datetime.utcnow())
2025/06/04 11:20:22 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://anshuopenai2.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:openai._base_client:Retrying request to /chat/compl

  0%|          | 0/1 [00:00<?, ?it/s]

2025/06/04 11:21:52 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/06/04 11:21:52 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.
2025/06/04 11:21:52 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for automated readability index metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/06/04 11:21:52 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'ari_grade_level' at index 3 in the `extra_metrics` parameter because it returned None.
2025/06/04 11:21:52 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'exact_match' at index 4 in the `extra_metrics` parameter because it returned None.


  0%|          | 0/4 [00:00<?, ?it/s]

{'toxicity/v1/mean': np.float64(0.0008192558816517703), 'toxicity/v1/variance': np.float64(2.1200623857396135e-07), 'toxicity/v1/p90': np.float64(0.001293126924429089), 'toxicity/v1/ratio': 0.0, 'professionalism/v1/mean': np.float64(5.0), 'professionalism/v1/variance': np.float64(0.0), 'professionalism/v1/p90': np.float64(5.0)}


In [37]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,toxicity/v1/score,professionalism/v1/score,professionalism/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The function designated as `useEffect()` withi...,349,0.000657,5,The response is written in an excessively form...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...","The designation of the keyword ""static"" when a...",119,0.000160,5,The response uses highly formal and precise la...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,"In the Python programming language, the `final...",248,0.001072,5,The response is written in an exceptionally fo...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,The distinction between multiprocessing and mu...,412,0.001388,5,The response uses highly formal and respectful...
